In [2]:
!pip uninstall tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import numpy as np
import tensorflow as tf
import os
import cv2
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision, F1Score
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

# Directories
drunk_dir = '/content/drive/MyDrive/NeurIPS/Code/cropped/drunk'
sober_dir = '/content/drive/MyDrive/NeurIPS/Code/cropped/sober'

# Load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label)
    return images, labels

drunk_images, drunk_labels = load_images_from_folder(drunk_dir, 1)  # Label 1 for drunk
sober_images, sober_labels = load_images_from_folder(sober_dir, 0)  # Label 0 for sober


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
drunk_images = drunk_images[:-(5409-4670)]
drunk_labels = drunk_labels[:-(5409-4670)]

print(len(drunk_images))
print(len(drunk_labels))
print(len(sober_labels))

allImages = []
allLabels = []

for i in range(len(drunk_images)):
  allImages.append(drunk_images[i])
  allImages.append(sober_images[i])
  allLabels.append(drunk_labels[i])
  allLabels.append(sober_labels[i])

# Combine and split the dataset
images = np.array(allImages)
labels = np.array(allLabels)

# Normalize images
images = images.astype('float32') / 255.0

4670
4670
4670


In [12]:
# Split the dataset
train_x, temp_x, train_y, temp_y = train_test_split(images, labels, test_size=0.3, random_state=42, shuffle = False)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.5, random_state=42, shuffle = False)

# Data generators
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(train_x, train_y, batch_size=32)
val_generator = val_datagen.flow(val_x, val_y, batch_size=32)
test_generator = test_datagen.flow(test_x, test_y, batch_size=32)

# Load Inception-ResNetV2 model
base_model = InceptionResNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base_model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)  # Binary classification (drunk or sober)

model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', Recall(), Precision()])
model.summary()

# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_609 (Conv2D)       │ (None, 111, 111, 32)   │            864 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_609   │ (None, 111, 111, 32)   │             96 │ conv2d_609[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_609            │ (None, 111, 111, 32)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_610 (Conv2D)       │ (None, 109, 109, 32)   │          9,216 │ activation_609[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_610   │ (None, 109, 109, 32)   │             96 │ conv2d_610[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_610            │ (None, 109, 109, 32)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_611 (Conv2D)       │ (None, 109, 109, 64)   │         18,432 │ activation_610[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_611   │ (None, 109, 109, 64)   │            192 │ conv2d_611[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_611            │ (None, 109, 109, 64)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_12          │ (None, 54, 54, 64)     │              0 │ activation_611[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_612 (Conv2D)       │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d_12[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_612   │ (None, 54, 54, 80)     │            240 │ conv2d_612[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_612            │ (None, 54, 54, 80)     │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_613 (Conv2D)  

 Total params: 55,124,193 (210.28 MB)

 Trainable params: 787,457 (3.00 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

In [13]:
history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc, test_recall, test_precision = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')
print(f'Test Recall: {test_recall}')
print(f'Test Precision: {test_precision}')

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


205/205 ━━━━━━━━━━━━━━━━━━━━ 135s 488ms/step - accuracy: 0.5750 - loss: 0.8477 - precision_1: 0.5782 - recall_1: 0.5874 - val_accuracy: 0.7116 - val_loss: 0.5836 - val_precision_1: 0.7043 - val_recall_1: 0.7304
Epoch 2/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 73s 345ms/step - accuracy: 0.6628 - loss: 0.6112 - precision_1: 0.6631 - recall_1: 0.6614 - val_accuracy: 0.7316 - val_loss: 0.5430 - val_precision_1: 0.7400 - val_recall_1: 0.7147
Epoch 3/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 73s 343ms/step - accuracy: 0.6878 - loss: 0.5894 - precision_1: 0.6904 - recall_1: 0.6777 - val_accuracy: 0.7016 - val_loss: 0.5696 - val_precision_1: 0.8734 - val_recall_1: 0.4722
Epoch 4/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 73s 343ms/step - accuracy: 0.7039 - loss: 0.5720 - precision_1: 0.7066 - recall_1: 0.6682 - val_accuracy: 0.7245 - val_loss: 0.5353 - val_precision_1: 0.8621 - val_recall_1: 0.5350
Epoch 5/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 73s 345ms/step - accuracy: 0.7118 - loss: 0.5653 - precision_1: 0.7199 - recall_1: 0.7